##### Bibliotecas


In [6]:
import os
import numpy as np
import pandas as pd
import openpyxl #biblioteca para ler o arquivo .xlsx
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
#from plotly.subplots import make_subplots

In [7]:
pio.renderers.default = 'notebook'  # Configura o renderizador para notebook

##### Globais

In [9]:
diretorio        = 'datasets'
arquivo_extensao = '.csv'

___
##### Conjunto de Dados - ANP

Série Histórica de Preços de Combustíveis e de GLP   
https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

A análise será realizada no **Conjunto de Dados semanais agrupados por semestre (2013 a 2023)**, considerando somente os **Combustíveis automotivos**, totalizando 21 arquivos csv.
<br><br>
___

##### Análise da Estrutura dos Dados (antes da limpeza)



In [68]:
df_dados          = pd.DataFrame()    
df_estrutra_dados = pd.DataFrame()
lista_arquivos    = []

# Lista os arquivos do diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(arquivo_extensao):
       lista_arquivos.append(arquivo) 
       
# Carrega o DataFrame 
for dataset in lista_arquivos:
    caminho_arquivo   = os.path.join(diretorio, dataset)
    df_dados          = pd.read_csv(caminho_arquivo, sep=';', low_memory=False)
    df_estrutra_dados = pd.concat([df_estrutra_dados, df_dados], ignore_index=True)

##### Dimensões da Base de Dados

In [ ]:
print(f"Linhas: {len(df_estrutra_dados)}")
print(f"Colunas: {len(df_estrutra_dados.iloc[0])}")

##### Estrutura da Base de Dados

In [ ]:
df_estrutra_dados.info()

##### Campos nulos

In [ ]:
df_estrutra_dados.isnull().sum()

##### Visualização do DataFrame (Head e Tail)

In [ ]:
print(display(df_estrutra_dados)) 

___
##### Análise da Estrutura dos Dados (processo de limpeza)

Otimização:
* Colunas não consideradas na análise serão excluídas do dataframe
* A fim de um melhor aproveitamento de espaço, algumas colunas terão o seu tipo alterado

In [5]:
df_dados        = pd.DataFrame()
df_dados_limpos = pd.DataFrame()
lista_arquivos  = []

# Lista os arquivos do diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(arquivo_extensao):
       lista_arquivos.append(arquivo) 

# Carrega o DataFrame 
for dataset in lista_arquivos:
    caminho_arquivo = os.path.join(diretorio, dataset)
    df_dados        = pd.read_csv(caminho_arquivo,
                                  sep=';', thousands = '.', decimal = ',',
                                  dtype = {'Regiao - Sigla': 'category',
                                           'Estado - Sigla': 'category',
                                           'Valor de Compra': np.float64})
    
    df_dados_limpos = pd.concat([df_dados_limpos, df_dados], ignore_index=True)

# Exclui colunas    
df_dados_limpos.drop(columns=['Nome da Rua',
                              'Numero Rua',
                              'Complemento',
                              'Bairro',
                              'Cep',
                              'Valor de Compra',
                              'Unidade de Medida'], inplace=True)

df_dados_limpos.rename(columns={'Regiao - Sigla': 'Regiao', 
                                'Estado - Sigla': 'Estado',
                                'CNPJ da Revenda': 'CNPJ',
                                'Data da Coleta': 'Data_Coleta',
                                'Valor de Venda': 'Valor_Venda'}, inplace=True)

# Criando colunas de mês e ano
df_dados_limpos['Data_Coleta'] = pd.to_datetime(df_dados_limpos['Data_Coleta'], format="%d/%m/%Y")
df_dados_limpos['Ano']         = df_dados_limpos['Data_Coleta'].dt.year
df_dados_limpos['Mes']         = df_dados_limpos['Data_Coleta'].dt.month
                     
df_dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10265783 entries, 0 to 10265782
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Regiao       category      
 1   Estado       category      
 2   Municipio    object        
 3   Revenda      object        
 4   CNPJ         object        
 5   Produto      object        
 6   Data_Coleta  datetime64[ns]
 7   Valor_Venda  float64       
 8   Bandeira     object        
 9   Ano          int32         
 10  Mes          int32         
dtypes: category(2), datetime64[ns](1), float64(1), int32(2), object(5)
memory usage: 646.2+ MB


##### Visualização do DataFrame (Head e Tail)

In [ ]:
print(display(df_dados_limpos)) 

___
##### Conjunto de Dados - IPCA

*Fonte: IBGE*  
https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?t=series-historicas&utm_source=landing&utm_medium=explica&utm_campaign=inflacao#plano-real-mes

<br>
Ao desflacionar os preços dos combustíveis usando o **IPCA**, estamos removendo o efeito da inflação e tornando os preços comparáveis em termos reais ao longo do tempo. Isso permite uma análise mais precisa das variações nos preços dos combustíveis, ajudando a identificar padrões, tendências e comportamentos que podem ser influenciados por fatores econômicos, políticos ou outros.
<br><br>

___

##### Análise da Estrutura dos Dados (antes da limpeza)

In [10]:
# LÊ o arquivo XLSX
arquivo_excel = diretorio + '/20230820091804.xlsx'
df_ipca       = pd.read_excel(arquivo_excel, thousands = '.', decimal = ',')

##### Visualização do DataFrame

In [26]:
display(df_ipca)
df_ipca.info()

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 341,Unnamed: 342,Unnamed: 343,Unnamed: 344,Unnamed: 345,Unnamed: 346,Unnamed: 347,Unnamed: 348,Unnamed: 349,Unnamed: 350
0,julho 1994,agosto 1994,setembro 1994,outubro 1994,novembro 1994,dezembro 1994,janeiro 1995,fevereiro 1995,março 1995,abril 1995,...,outubro 2022,novembro 2022,dezembro 2022,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023
2,6.84,1.86,1.53,2.62,2.81,1.71,1.70,1.02,1.55,2.43,...,0.59,0.41,0.62,0.53,0.84,0.71,0.61,0.23,-0.08,0.12


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 2
Columns: 349 entries, Unnamed: 2 to Unnamed: 350
dtypes: object(349)
memory usage: 5.5+ KB


___
##### Análise da Estrutura dos Dados (processo de limpeza)

In [12]:
# Seleciona a área útil do arquivo (intervalo de células C2:MN4)
df_ipca = df_ipca.iloc[0:3, 2:351]  # Linhas 1 a 3, colunas C a MM
display(df_ipca)

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 341,Unnamed: 342,Unnamed: 343,Unnamed: 344,Unnamed: 345,Unnamed: 346,Unnamed: 347,Unnamed: 348,Unnamed: 349,Unnamed: 350
0,julho 1994,agosto 1994,setembro 1994,outubro 1994,novembro 1994,dezembro 1994,janeiro 1995,fevereiro 1995,março 1995,abril 1995,...,outubro 2022,novembro 2022,dezembro 2022,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.84,1.86,1.53,2.62,2.81,1.71,1.70,1.02,1.55,2.43,...,0.59,0.41,0.62,0.53,0.84,0.71,0.61,0.23,-0.08,0.12


In [125]:
# Exclui linha sem registro 
df_ipca = df_ipca.dropna(how='all')
display(df_ipca)

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 341,Unnamed: 342,Unnamed: 343,Unnamed: 344,Unnamed: 345,Unnamed: 346,Unnamed: 347,Unnamed: 348,Unnamed: 349,Unnamed: 350
0,julho 1994,agosto 1994,setembro 1994,outubro 1994,novembro 1994,dezembro 1994,janeiro 1995,fevereiro 1995,março 1995,abril 1995,...,outubro 2022,novembro 2022,dezembro 2022,janeiro 2023,fevereiro 2023,março 2023,abril 2023,maio 2023,junho 2023,julho 2023
2,6.84,1.86,1.53,2.62,2.81,1.71,1.70,1.02,1.55,2.43,...,0.59,0.41,0.62,0.53,0.84,0.71,0.61,0.23,-0.08,0.12


In [81]:
# Inverte as linhas com as colunas: Transpõe
df_ipca_T=df_ipca.transpose()
# Refaz o índice existente
df_ipca_T.reset_index(inplace=True, drop=True) 

# Renomeia as colunas
df_ipca_T.columns = ['Mes_Ano', 'IPCA']
# Altera o tipo de dado do campo IPCA para float64
df_ipca_T['IPCA'] = df_ipca_T['IPCA'].astype(float)

display(df_ipca_T)
df_ipca_T.info()

,Mes_Ano,IPCA
0,julho 1994,6.84
1,agosto 1994,1.86
2,setembro 1994,1.53
3,outubro 1994,2.62
4,novembro 1994,2.81
...,...,...
344,março 2023,0.71
345,abril 2023,0.61
346,maio 2023,0.23
347,junho 2023,-0.08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Mes_Ano  349 non-null    object 
 1   IPCA     349 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.6+ KB


In [82]:
# Mapeando nomes dos meses para números
meses_dict = {
    'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4, 'maio': 5, 'junho': 6,
    'julho': 7, 'agosto': 8, 'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
}

# Convertendo a coluna 'MesAno' em formato de data
df_ipca_T['Mes_Ano'] = df_ipca_T['Mes_Ano'].apply(lambda x: pd.to_datetime(f"{meses_dict[x.split()[0].lower()]} {x.split()[1]}"))

# Exibindo o DataFrame resultante
display(df_ipca_T)
df_ipca_T.info()

,Mes_Ano,IPCA
0,1994-07-01,6.84
1,1994-08-01,1.86
2,1994-09-01,1.53
3,1994-10-01,2.62
4,1994-11-01,2.81
...,...,...
344,2023-03-01,0.71
345,2023-04-01,0.61
346,2023-05-01,0.23
347,2023-06-01,-0.08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Mes_Ano  349 non-null    datetime64[ns]
 1   IPCA     349 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.6 KB


In [123]:
# Criando colunas de mês e ano
df_ipca_T['Ano'] = df_ipca_T['Mes_Ano'].dt.year
df_ipca_T['Mes'] = df_ipca_T['Mes_Ano'].dt.month

display(df_ipca_T)
df_ipca_T.info()

,Mes_Ano,IPCA,Ano,Mes
0,1994-07-01,6.84,1994,7
1,1994-08-01,1.86,1994,8
2,1994-09-01,1.53,1994,9
3,1994-10-01,2.62,1994,10
4,1994-11-01,2.81,1994,11
...,...,...,...,...
344,2023-03-01,0.71,2023,3
345,2023-04-01,0.61,2023,4
346,2023-05-01,0.23,2023,5
347,2023-06-01,-0.08,2023,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Mes_Ano  349 non-null    datetime64[ns]
 1   IPCA     349 non-null    float64       
 2   Ano      349 non-null    int32         
 3   Mes      349 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(2)
memory usage: 8.3 KB
